there are two type:
1)new user_id
2)continuation of last user_id

* timestamp_diff
* no_of_question_answered
* no_of_question_correct_answered
* moving_Avg_prior_index
* last , Prior Question had explained

In [1]:
import seaborn as sns
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import gc
import psutil
import random
import torch
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as layers
import tensorflow.keras.models as Models
import pandas as pd
def seed_everything(seed): 
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
seed_everything(42)

from threading import Thread 

In [2]:
# import warnings
# warnings. simplefilter(action='ignore', category='SettingWithCopyWarning') 

In [3]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns: #columns
        col_type = df[col].dtypes
        if col_type in numerics: #numerics
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
gc.collect()
train=pd.read_csv('../input/pipeline/train.csv')
train=reduce_mem_usage(train, verbose=True)
train_user_id=train.user_id.unique()
data=pd.read_csv('../input/pipeline/data.csv')
data=reduce_mem_usage(data, verbose=True)
gc.collect()
#####################
average_answer_question=pd.read_csv('../input/final-preprocessing/average_answer_question.csv')
question=pd.read_csv('../input/final-preprocessing/question.csv')
average_answer_question=average_answer_question.set_index('content_id').round(0).astype(int)
question=question.set_index('content_id').round(0).astype(int)
##########################################
train.drop(columns=['Day_passed','timestamp_diff','Percent_of_student_Answered_Correctly'],inplace=True)
train['no_of_question_answered']=train['no_of_question_answered'].astype(int).values
array=(train['moving_average_index'].values.round(2)*100).astype(int)
train.drop(columns=['moving_average_index'],inplace=True)
train['moving_average_index']=array/100
train['Avg_prior_question_elapsed_time']=train['Avg_prior_question_elapsed_time'].values.astype(int)
train['no_of_question_correct_answered']=train['no_of_question_correct_answered'].values+train['answered_correctly'].values
train.drop(columns=['answered_correctly'],inplace=True)

Mem. usage decreased to 21.06 Mb (62.5% reduction)
Mem. usage decreased to  5.25 Mb (56.2% reduction)


In [5]:
train

,user_id,Avg_prior_question_elapsed_time,no_of_question_correct_answered,moving_explaination,no_of_question_answered,timestamp,moving_average_index
0,115,22476,31,6,43,668090043,1.01
1,115,23037,31,6,44,668090043,1.01
2,115,23160,32,6,45,668090043,1.00
3,124,56297,7,0,26,571323,0.95
4,124,54084,7,0,27,571323,0.92
...,...,...,...,...,...,...,...
669092,2147470777,25478,520,720,750,13332685203,0.81
669093,2147470777,25352,520,721,751,13332685203,0.81
669094,2147481750,15882,38,39,49,55954768,0.92
669095,2147482216,17196,177,266,274,1470594073,1.00


**prior_group_responses (option)<br>**

In [6]:
train_1=train.copy()
train_1=train_1.reset_index()
train_1=train_1.loc[train_1.groupby(['user_id'])['index'].max().values].reset_index(drop=True).drop(columns=['index'])
train_1=train_1[['user_id','no_of_question_answered','no_of_question_correct_answered','moving_explaination']]
train.drop(columns=['no_of_question_answered','no_of_question_correct_answered','moving_explaination'],inplace=True)
data.drop(columns=['start_index','end_index'],inplace=True)
train_1['no_of_question_answered']=train_1.no_of_question_answered.astype(int)
train_1['no_of_question_correct_answered']=np.ceil(train_1['no_of_question_correct_answered'].values).astype(int)

In [7]:
train_1

,user_id,no_of_question_answered,no_of_question_correct_answered,moving_explaination
0,115,45,32,6
1,124,29,7,0
2,2746,18,11,11
3,5382,124,84,113
4,8623,108,70,96
...,...,...,...,...
393562,2147470770,225,173,195
393563,2147470777,751,520,721
393564,2147481750,49,38,39
393565,2147482216,274,177,266


In [8]:
def basic_pipeline(test):
    test=test.reset_index()
    test['task_container_id'].fillna(0,inplace=True)
    test['timestamp'].fillna(0,inplace=True)
    test['content_id'].fillna(0,inplace=True)
    test['user_id'].fillna(0,inplace=True)
    test['prior_question_had_explanation'].replace({np.NaN:0,True:1,False:0},inplace=True)
    test['prior_question_elapsed_time'].fillna(-1,inplace=True)
    test['prior_question_elapsed_time']=test['prior_question_elapsed_time'].astype(int) 
    test.loc.__setitem__((test.timestamp==0, ('prior_question_elapsed_time')),-2)
    test.drop(columns=['content_type_id','prior_group_responses'],inplace=True)
    test['prior_group_answers_correct']=test.prior_group_answers_correct.fillna(-1).values.astype(str)
    ###########
    array=[eval(row) for row in test.prior_group_answers_correct.values]
    value=np.array([[[len(i),np.sum(i)] if(len(i)>0) else [0,0]][0] if(type(i)==list) else [[0,0]][0]  for i in array ])
    #value
    test['no_of_question_answered']=value[:,0]
    test['no_of_question_correct_answered']=value[:,1]
    test=test.merge(average_answer_question,on='content_id',how='left')
    test=test.merge(question,on='content_id',how='left')
    test.drop(columns=['group_num','row_id','content_id','prior_group_answers_correct'],inplace=True)
    return test

In [9]:
# test=pd.read_csv('../input/riiid-test-env/test_data.csv')
# test=basic_pipeline(test)

Here it's content_type_id is always zero<br>
same task_container_id come at  time<br>

In [10]:
columns=train_1.columns#train.values
Dict={col:i for i,col in enumerate(columns)}
train_1=train_1.values
Dict

{'user_id': 0,
 'no_of_question_answered': 1,
 'no_of_question_correct_answered': 2,
 'moving_explaination': 3}

In [11]:
train_1

array([[       115,         45,         32,          6],
       [       124,         29,          7,          0],
       [      2746,         18,         11,         11],
       ...,
       [2147481750,         49,         38,         39],
       [2147482216,        274,        177,        266],
       [2147482888,         26,         15,         19]])

In [12]:
def feature_1_pipeline(test):
    global train_1
    global train_user_id
    test_user_id=test.user_id.unique()
    array_1=np.zeros(test.shape[0])
    array_2=array_1.copy()
    array_3=array_1.copy()
    for ID in test_user_id:
        temp=test[test.user_id==ID]
        index=np.array(temp.index)
        temp=temp.values
        cnt=temp.shape[0]
        temp=np.cumsum(temp[:,4])
        if(ID in train_user_id):
            data_point=train_1[train_1[:,0]==ID][-1]
            no_of_question_answered,no_of_question_correct_answered,moving_exp=data_point[1],data_point[2],data_point[3]
            train_1[train_1[:,0]==ID,1]=no_of_question_answered+cnt
            #train_1[train_1[:,0]==ID,-1]=moving_exp+temp[-1]
        else:
            no_of_question_answered=0
            no_of_question_correct_answered=0
            moving_exp=0
            #train_1=np.insert(train_1,0,[ID,cnt-1,0,temp[-1]],axis=0)
            
        no_of_question_answered=np.arange(0,cnt)+no_of_question_answered
        moving_prior_exp=temp+moving_exp
        #update
        array_1[index]=no_of_question_answered
        array_2[index]=no_of_question_correct_answered
        array_3[index]=moving_prior_exp
    test['no_of_question_answered']=test['no_of_question_answered'].values+array_1.astype(int)
    test['no_of_question_correct_answered']=array_2+test['no_of_question_correct_answered'].values
    test['moving_explaination']=array_3
    
    return test

In [13]:
#-1 moving average index can make problem sorted
def feature_2_pipeline(test):
    global data,train,train_user_id
    user_id=test.user_id.values
    test_user_id=np.unique(user_id)
    count=test.groupby(['user_id','task_container_id'],sort=False)['Avg_prior_question_elapsed_time'].count().values
    temp_=test.groupby(['user_id','task_container_id'],sort=False).mean().reset_index()
    temp_['count']=count
    temp_=temp_[['user_id','timestamp','prior_question_elapsed_time','Avg_prior_question_elapsed_time','count']].values
    ########################
    array_1=np.zeros(test.shape[0])
    array_2=array_1.copy()
    for ID in test_user_id:
        temp=temp_[temp_[:,0]==ID].copy()
        cnt=temp.shape[0]
        Moving_average_index=[]
        Timestamp_diff=[]  
        #average of train
        if(ID in train_user_id):
            mvr,timestamp,avg=train[train[:,0]==ID,[3,2,1]].copy()
            distance=data[data[:,0]==ID,1].copy()
            timestamp_diff=(temp[0,1]-timestamp)/temp[0,-1]
            if(temp[0,2]<0):
                temp[0,2]=timestamp_diff
            moving_average_index=((temp[0,2]/avg)*np.arange(1,temp[0,-1]+1)+mvr*distance)/(distance+np.arange(1,temp[0,-1]+1))
            distance+=int(temp[0,-1])
            Moving_average_index.append(moving_average_index)
            Timestamp_diff.append([timestamp_diff]*int(temp[0,-1]))
            moving_average_index_=np.mean(moving_average_index)
        else:
            distance=0
            Moving_average_index.append([-2]*int(temp[0,-1]))
            Timestamp_diff.append([0]*int(temp[0,-1]))
            moving_average_index_=-2
            train=np.insert(train,0,[ID,temp[0,3],0,-2],axis=0)
            data=np.insert(data,0,[ID,0],axis=0)
        for i  in range(1,cnt):
            if(temp[i,2]<0):
                temp[i,2]=timestamp_diff
            moving_average_index=((temp[i,2]/temp[i-1,3])*np.arange(1,temp[i,-1]+1)+moving_average_index_*distance)/(distance+np.arange(1,temp[i,-1]+1))
            timestamp_diff=(temp[i,1]-temp[i-1,1])/temp[i,-1]
            Moving_average_index.append(moving_average_index)
            Timestamp_diff.append([timestamp_diff]*int(temp[i,-1]))
            moving_average_index_=np.mean(moving_average_index)
            distance+=int(temp[i,-1])
        ########
        train[train[:,0]==ID,3]=moving_average_index_
        train[train[:,0]==ID,2]=temp[-1,1]
        data[data[:,0]==ID,1]=distance
        ######
        array_1[user_id==ID]=np.hstack(Moving_average_index)
        array_2[user_id==ID]=np.hstack(Timestamp_diff)
    test['timestamp_diff']=array_2.astype(int)
    test['moving_average_index']=array_1
    return test

In [14]:
{'task_container_id': 0,
 'timestamp': 1,
 'user_id': 2,
 'prior_question_elapsed_time': 3,
 'no_of_question_answered': 4,
 'no_of_question_correct_answered': 5,
 'Avg_prior_question_elapsed_time': 6,
 'Percent_of_student_Answered_Correctly': 7}

{'task_container_id': 0,
 'timestamp': 1,
 'user_id': 2,
 'prior_question_elapsed_time': 3,
 'no_of_question_answered': 4,
 'no_of_question_correct_answered': 5,
 'Avg_prior_question_elapsed_time': 6,
 'Percent_of_student_Answered_Correctly': 7}

In [15]:
train=train.groupby(['user_id']).mean().reset_index()
columns=train.columns#train.values
Dict={col:i for i,col in enumerate(columns)}
train=train.values
print(Dict)

{'user_id': 0, 'Avg_prior_question_elapsed_time': 1, 'timestamp': 2, 'moving_average_index': 3}


In [16]:
#71,45.9,0

In [17]:
data=data.values

In [18]:
{'user_id': 0,
 'Avg_prior_question_elapsed_time': 1,
 'timestamp': 2,
 'moving_average_index': 3}

{'user_id': 0,
 'Avg_prior_question_elapsed_time': 1,
 'timestamp': 2,
 'moving_average_index': 3}

In [19]:
#model
import os
Dir='../input/neural-network'
file=[]
for f in os.listdir(Dir) :
    if('.h5' in str(f)):
        file.append(Dir+'/'+f)
        
import os
Dir='../input/riiid-model-2-6-1'
file_1=[]
for f in os.listdir(Dir) :
    if('.h5' in str(f)):
        file_1.append(Dir+'/'+f)

In [20]:
import tensorflow_addons as tfa
def model():
    Input=layers.Input((8,))
    x=layers.BatchNormalization()(Input)
    x=tfa.layers.WeightNormalization(layers.Dense(128,activation='relu'))(x)
    x=layers.BatchNormalization()(x)
    x=layers.Dropout(0.2)(x)
    x=tfa.layers.WeightNormalization(layers.Dense(64,activation='relu'))(x)
    x=layers.BatchNormalization()(x)
    x=layers.Dropout(0.1)(x)
    x=tfa.layers.WeightNormalization(layers.Dense(32,activation='relu'))(x)
    x=layers.BatchNormalization()(x)
    x=layers.Dropout(0.1)(x)
    x=tfa.layers.WeightNormalization(layers.Dense(16,activation='relu'))(x)
    x=layers.BatchNormalization()(x)
    #x=layers.Dropout(0.1)(x)
    x=tfa.layers.WeightNormalization(layers.Dense(1,activation='sigmoid'))(x)
    model_neural=Models.Model(Input,x)
    ###################
    bce = tf.keras.losses.BinaryCrossentropy()
    optimizer=tf.keras.optimizers.Adam(lr=0.1)# if optimizer=='Adam' else tf.keras.optimizers.SGD()
    model_neural.compile(loss=bce,optimizer=optimizer,metrics=[tf.keras.metrics.AUC()])
    return model_neural

In [21]:
features=['Day_passed', 'timestamp_diff', 'Avg_prior_question_elapsed_time','Percent_of_student_Answered_Correctly','no_of_question_correct_answered', 'moving_explaination', 'no_of_question_answered', 'moving_average_index']

In [22]:
mean_std=np.load('../input/sampling-points-50lakh/scaler.npy')
mean_std

array([['Day_passed', '31.0', '134.30762356378116'],
       ['timestamp_diff', '44631.0', '166269.07731776647'],
       ['Avg_prior_question_elapsed_time', '25145.262497211374',
        '10276.091729464224'],
       ['Percent_of_student_Answered_Correctly', '0', '100'],
       ['no_of_question_correct_answered', '0', '1'],
       ['moving_explaination', '0', '1'],
       ['no_of_question_answered', '488.0', '1835.759926350772'],
       ['moving_average_index', '0.0', '26.328125']], dtype='<U37')

In [23]:
gc.collect()

68

In [24]:
model_neural_1=model()
model_neural_2=model()
model_neural_3=model()
model_neural_4=model()
model_neural_5=model()
model_neural_6=model()

model_neural_1.load_weights(file[0])
model_neural_2.load_weights(file[1])
model_neural_3.load_weights(file[2])
model_neural_4.load_weights(file_1[0])
model_neural_5.load_weights(file_1[1])
model_neural_6.load_weights(file_1[2])
def model_prediction(test_df):
    global mean_std,features,file,model_neural_1,model_neural_2,model_neural_3,model_neural_4
    prediction=[]
    temp_test=test_df[features].copy().values
    t=temp_test[:,6].copy()
    t[t==0]=1
    temp_test[:,4]=temp_test[:,4]/t
    temp_test[:,5]=temp_test[:,5]/t
    #print(mean_std)
    for i,(_,mean,std) in enumerate(mean_std):
        temp_test[:,i]=(temp_test[:,i]-float(mean))/float(std)
    prediction=[]
    pred1=model_neural_1(temp_test)
    pred2=model_neural_2(temp_test)
    pred3=model_neural_3(temp_test)
    pred4=model_neural_4(temp_test)
    pred5=model_neural_5(temp_test)
    pred6=model_neural_6(temp_test)
    #prediction.append([pred1,pred2,pred3,pred4,pred5,pred6])
    prediction=np.array(pred1+pred2+pred3+pred4+pred5+pred6)/6
    prediction=prediction.reshape(-1,)
    prediction=np.round(prediction,2)
    return prediction

In [25]:
gc.collect()
gc.collect()

0

In [26]:
%%time
import riiideducation
import time
env = riiideducation.make_env()
iter_test = env.iter_test()
start_=time.process_time()
for test_df, sample_prediction_df in tqdm(iter_test):
    if(len(sample_prediction_df)<=1): 
        env.predict(sample_prediction_df)
        continue
    test_df=test_df.loc[test_df['content_type_id'] == 0]
    #####
    start=time.process_time()
    test_df=basic_pipeline(test_df.copy())
    ######################
    #fist pipeline
    test_df=feature_1_pipeline(test_df.copy())
    test_df=feature_2_pipeline(test_df.copy())
    test_df['Day_passed']=(test_df['timestamp'].values/(24*60*60*1000)).astype(int)
    test_df.loc.__setitem__((test_df['timestamp_diff'].values>=10*60*1000, ('timestamp_diff')),10*60*1000)
    #################################################################
    prediction=model_prediction(test_df)
    sample_prediction_df['answered_correctly']=prediction
    #update for first pipeline
    test_df['no_of_question_correct_answered']=test_df['no_of_question_correct_answered'].values+prediction
    temp=test_df.groupby('user_id',sort=False)[['no_of_question_answered','no_of_question_correct_answered','moving_explaination']].max().reset_index()
    train_1=pd.concat([pd.DataFrame(train_1,columns=['user_id', 'no_of_question_answered','no_of_question_correct_answered','moving_explaination']),temp]).groupby('user_id',sort=False).max().reset_index().values
    env.predict(sample_prediction_df)
    train_user_id=train_1[:,0].copy()
    del temp,sample_prediction_df,test_df
print(time.process_time()-start_)

0it [00:00, ?it/s]

0.19402121599999944
CPU times: user 194 ms, sys: 6.02 ms, total: 200 ms
Wall time: 258 ms


In [27]:
# %%time
# import riiideducation
# import time
# env = riiideducation.make_env()
# iter_test = env.iter_test()
# #start=time.process_time()
# for test_df, sample_prediction_df in tqdm(iter_test):
#     if(len(sample_prediction_df)<=1): 
#         env.predict(sample_prediction_df)
#         continue
#     ###########
#     start=time.process_time()
#     test_df=test_df.loc[test_df['content_type_id'] == 0]
#     print(time.process_time()-start)#1
#     start=time.process_time()
#     test_df=basic_pipeline(test_df.copy())
#     print(time.process_time()-start)#2
#     start=time.process_time()
#     ######################
#     test_df=feature_1_pipeline(test_df.copy())
#     print(time.process_time()-start)#3
#     start=time.process_time()
#     test_df=feature_2_pipeline(test_df.copy())
#     print(time.process_time()-start)#4
#     start=time.process_time()
#     test_df['Day_passed']=(test_df['timestamp'].values/(24*60*60*1000)).astype(int)
#     print(time.process_time()-start)#5
#     start=time.process_time()
#     test_df.loc.__setitem__((test_df['timestamp_diff'].values>=10*60*1000, ('timestamp_diff')),10*60*1000)
#     print(time.process_time()-start)#6
#     start=time.process_time()
#     #################################################################
#     prediction=model_prediction(test_df)
#     print(time.process_time()-start)#7
#     start=time.process_time()
#     sample_prediction_df['answered_correctly']=prediction
#     start=time.process_time()
#     #update for first pipeline
#     start=time.process_time()
#     test_df['no_of_question_correct_answered']=test_df['no_of_question_correct_answered'].values+prediction
#     print(time.process_time()-start)#8
#     start=time.process_time()
#     temp=test_df.groupby('user_id',sort=False)[['no_of_question_answered','no_of_question_correct_answered','moving_explaination']].max().reset_index()
#     print(time.process_time()-start)#9
#     start=time.process_time()
#     train_1=pd.concat([pd.DataFrame(train_1,columns=['user_id', 'no_of_question_answered','no_of_question_correct_answered','moving_explaination']),temp]).groupby('user_id',sort=False).max().reset_index().values
#     print(time.process_time()-start)#10
#     start=time.process_time()
#     env.predict(sample_prediction_df)
#     print(time.process_time()-start)#11
#     start=time.process_time()
#     print(time.process_time()-start)#12
#     start=time.process_time()
#     train_user_id=train_1[:,0].copy()
#     del temp,sample_prediction_df,test_df
#     print(time.process_time()-start)#
#     print('\n')
# #print(time.process_time()-start)

In [28]:
#     test_df['no_of_question_correct_answered']=test_df['no_of_question_correct_answered'].values+prediction
#     temp=test_df.groupby('user_id',sort=False)[['no_of_question_answered','no_of_question_correct_answered','moving_explaination']].max().reset_index()
#     train_1=pd.concat([pd.DataFrame(train_1,columns=['user_id', 'no_of_question_answered','no_of_question_correct_answered','moving_explaination']),temp]).groupby('user_id',sort=False).max().reset_index().values
#     env.predict(sample_prediction_df)
#     train_user_id=train_1[:,0].copy()
#     del temp,sample_prediction_df,test_df

In [29]:
gc.collect()

60

In [30]:
data

array([[ 275030867,          0],
       [       115,         44],
       [       124,         28],
       ...,
       [2147481750,         48],
       [2147482216,        273],
       [2147482888,         25]], dtype=int32)

In [31]:
train

array([[ 2.75030867e+08,  1.87660000e+04,  0.00000000e+00,
        -2.00000000e+00],
       [ 1.15000000e+02,  2.28910000e+04,  6.68090043e+08,
         1.00666667e+00],
       [ 1.24000000e+02,  5.56582500e+04,  5.71323000e+05,
         9.10000000e-01],
       ...,
       [ 2.14748175e+09,  1.58820000e+04,  5.59547680e+07,
         9.20000000e-01],
       [ 2.14748222e+09,  1.71960000e+04,  1.47059407e+09,
         1.00000000e+00],
       [ 2.14748289e+09,  2.14560000e+04,  4.28692118e+08,
         1.24000000e+00]])

In [32]:
features

['Day_passed',
 'timestamp_diff',
 'Avg_prior_question_elapsed_time',
 'Percent_of_student_Answered_Correctly',
 'no_of_question_correct_answered',
 'moving_explaination',
 'no_of_question_answered',
 'moving_average_index']

In [33]:
train_1

array([[       115,         45,         32,          6],
       [       124,         29,          7,          0],
       [      2746,         18,         11,         11],
       ...,
       [2147481750,         49,         38,         39],
       [2147482216,        274,        177,        266],
       [2147482888,         26,         15,         19]])

In [34]:
pd.DataFrame(train)

,0,1,2,3
0,2.750309e+08,18766.000000,0.000000e+00,-2.000000
1,1.150000e+02,22891.000000,6.680900e+08,1.006667
2,1.240000e+02,55658.250000,5.713230e+05,0.910000
3,2.746000e+03,17372.000000,8.354570e+05,1.000000
4,5.382000e+03,28331.000000,2.101551e+09,1.660000
...,...,...,...,...
393563,2.147471e+09,22127.000000,2.832089e+09,1.070000
393564,2.147471e+09,25150.666667,1.333269e+10,0.810000
393565,2.147482e+09,15882.000000,5.595477e+07,0.920000
393566,2.147482e+09,17196.000000,1.470594e+09,1.000000


# After Prediction update the no_correctly_answered

In [35]:
pd.read_csv('./submission.csv')

FileNotFoundError: [Errno 2] No such file or directory: './submission.csv'